In [7]:
#import libraries
import sys
from os.path import dirname, join as pjoin
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.linalg as la
from statistics import mean 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
np.set_printoptions(threshold=1000)

In [8]:
#load heart data into arr
heart_data_dir= "heart.mat"
heart_data = sio.loadmat(heart_data_dir)
arr = heart_data['dat']

In [9]:
#load the labels describing presence or absence of disease
labels = heart_data['label']

In [58]:
def featureSubsetScore(arr, labels):   
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression(solver="liblinear")
    trainScores = [0]*10
    testScores = [0]*10
    nTrain = 200
    nTest = 70
    
    for i in range(0,10):
        random.seed(i+1)
        
        #initialize randomMatrix from original matrix
        randomArr = arr.copy()
        randomLabels = labels.copy();

        #randomize by swapping rows in randomMatrix
        randomIndices = list(range(0,270))
        random.shuffle(randomIndices)
        for j in range(0,270):            
            temp = randomArr[j]
            randomArr[j] = randomArr[randomIndices[j]]
            randomArr[randomIndices[j]] = temp
            
            temp = randomLabels[j]
            randomLabels[j] = randomLabels[randomIndices[j]]
            randomLabels[randomIndices[j]] = temp
            
        trainingArr = randomArr[range(0,nTrain),:]
        trainingLabels = randomLabels[range(0,nTrain)]       
        logreg.fit(trainingArr,trainingLabels.ravel())
       
        testingArr = randomArr[range(nTrain, nTrain+nTest),:]    
        testingLabels = randomLabels[range(nTrain, nTrain+nTest)].ravel()
        y_pred_train=logreg.predict(trainingArr)
        y_pred_test=logreg.predict(testingArr)
        
        nTrainCorrect = 0        
        for j in range(0,nTrain):
            if y_pred_train[j] == trainingLabels[j]:
                nTrainCorrect= nTrainCorrect + 1            
        trainScores[i] = nTrainCorrect/nTrain
        
        nTestCorrect = 0        
        for j in range(0,nTest):
            if y_pred_test[j] == testingLabels[j]:
                nTestCorrect = nTestCorrect +1            
        testScores[i] = nTestCorrect/nTest
    
    trainScoresMean = mean(trainScores)
    testScoresMean = mean(testScores)
    return [trainScoresMean, testScoresMean]

In [59]:
k=13
scoreBest = 0
selectedFeatures = []
allIndices = list(range(0,13))
for i in range(0,1000):
    random.seed(i+1)
    selectedIndices = random.sample(allIndices, k)
    trialFeatures = arr[:,selectedIndices]
    trainScore, testScore = featureSubsetScore(trialFeatures, labels)
    if testScore >scoreBest:
        scoreBest = testScore
        selectedFeatures = selectedIndices
print(scoreBest) 
print(selectedFeatures)
    
    
    

0.8314285714285714
[2, 9, 1, 4, 10, 7, 6, 3, 5, 11, 0, 12, 8]
